In [197]:
import numpy as np
import random

In [198]:
class Module():
    def __init__(self,module):
        module = module.split('|')
        self.name = module[0]
        self.staff = module[1]
        self.labs = module[2]
        self.constraints = module[3].split(',')
        
    def isin(self,mod):
        p

f = open('Modules.txt','r')
modules = []
line = f.readline()
while line:
    modules.append(Module(line))
    line = f.readline()
f.close()

In [199]:
def generate_random_timetable(modules):
    timetable = np.full((20,3),Module("None|None|None|None,None"),dtype=Module)
    totalmodules = modules
    
    for i in modules:
        totalmodules = totalmodules + ([i]*int(i.labs))
        
    k = 0
    for i in range(len(timetable)):
        for j in range(len(timetable[i])):
            timetable[i][j] = totalmodules[k]
            k = k+1
            if k>=len(totalmodules):
                break
        if k>=len(totalmodules):
                break
                
    timetable = timetable.flatten()
    np.random.shuffle(timetable)
    timetable = np.reshape(timetable,(20,3))
    return timetable



In [200]:
def checkConcurrence(li):
    count = 0
    #Checks if module is against another module that clashes.
    if li[0].name in li[1].constraints or li[0].name in li[2].constraints:
        count = count + 1
    if li[1].name in li[2].constraints or li[1].name in li[0].constraints:
        count = count + 1
    if li[2].name in li[0].constraints or li[2].name in li[1].constraints:
        count = count + 1
    
    #Checks if one module is being held at the same time in different rooms.
    if li[0].name==li[1].name:
        count = count + 1
    if li[1].name==li[2].name:
        count = count + 1
    if li[2].name==li[0].name:
        count = count + 1
    
    return count

def checkPrecedence(li):
    count = 0
    for i in range(len(li)):
        lecture = li[i][0].name
        if lecture == None:
            continue
        for j in range(0,i):
            if lecture == li[j][1].name:
                count = count + 1
            if lecture == li[j][2].name:
                count = count + 1
    return count


def fitness(timetable):
    precedence = checkPrecedence(timetable)
    
    concurrence = 0
    for i in timetable:
        concurrence = concurrence + checkConcurrence(i)
    
    return precedence * concurrence
        
timetable = generate_random_timetable(modules)
print(fitness(timetable))

817


In [224]:
def compare_fitness(child,parent):
    return child<=parent

In [202]:
def ruin_and_recreate(timetable):
    timetable = generate_random_timetable(modules)
    return timetable

In [203]:
def session_replace(timetable):
    row = np.random.randint(0,20,2)
    col = np.random.randint(0,3,2)
    t = timetable[row[0]][col[0]]
    timetable[row[0]][col[0]] = timetable[row[1]][col[1]]
    timetable[row[1]][col[1]] = t
    return timetable

In [261]:
def hillclimber(Niter, problem, D, generate, mutation, comparison):
    #comparison is to compare fitness
    #problem is fitness
    parent = generate(D)
    parent_fitness = problem(parent)
    fitness_list = [parent_fitness]
    
    for i in range(Niter):
        #Mutate
        child = mutation(parent)
        
        #Evaluate
        child_fitness = problem(child)
        
        #Pick next parent solution
        if comparison(child_fitness,parent_fitness):
            parent = child
            parent_fitness = child_fitness
            fitness_list.append(child_fitness)
        else:
            fitness_list.append(parent_fitness)
            
    return parent, fitness_list

optimised_timetable, fitnesslist = hillclimber(500,fitness,modules,generate_random_timetable,session_replace,compare_fitness)

print(fitnesslist)

# for i in optimised_timetable:
#     for j in i:
#         print(j.name)
#     print("===")

[1271, 1209, 1170, 1092, 1053, 984, 960, 960, 960, 960, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 882, 860, 860, 860, 860, 860, 860, 860, 860, 860, 860, 858, 814, 814, 814, 720, 720, 720, 720, 720, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561,